In [1]:
import sounddevice as sd
import numpy as np
import whisper
import queue
import torch

# Параметры записи
SAMPLERATE = 16000  # Частота дискретизации (Whisper требует 16kHz)
CHANNELS = 1        # Один канал (моно)
CHUNK_DURATION = 5  # Длина каждого куска в секундах
OVERLAP = 2         # Перекрытие между кусками в секундах
MODEL_SIZE = "medium"  # Можно "tiny", "base", "small", "medium", "large"
LANGUAGE = "ru"  # Язык распознавания

# Очередь для хранения аудиоданных
audio_queue = queue.Queue()

# Загрузка модели
print("Загрузка модели Whisper...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model(MODEL_SIZE, device=device)
print("Модель загружена!")

Загрузка модели Whisper...
Модель загружена!


In [2]:
# Функция обработки аудио в фоне
def callback(indata, frames, time, status):
    """Функция обратного вызова, получает аудиоданные"""
    if status:
        print(status)
    # Добавляем новый кусок в очередь
    audio_queue.put(indata.copy())

# Запускаем потоковую запись
with sd.InputStream(samplerate=SAMPLERATE, channels=CHANNELS, dtype='float32', callback=callback, blocksize=int(SAMPLERATE * (CHUNK_DURATION - OVERLAP))):
    print("Началась запись. Нажмите Ctrl+C для остановки.")
    
    # Кольцевой буфер для хранения аудиоданных
    ring_buffer = np.zeros(int(SAMPLERATE * CHUNK_DURATION), dtype=np.float32)
    
    try:
        while True:
            # Получаем новый кусок
            new_chunk = audio_queue.get()
            new_chunk = new_chunk[:, 0]  # Убираем лишнее измерение (канал)
            
            # Обновляем кольцевой буфер (сдвигаем влево, добавляем новый кусок справа)
            ring_buffer[:-len(new_chunk)] = ring_buffer[len(new_chunk):]
            ring_buffer[-len(new_chunk):] = new_chunk

            # Передаем в Whisper (Whisper сам обработает тишину)
            print("Обработка аудио...")
            result = model.transcribe(ring_buffer, language=LANGUAGE)
            print("📝 Распознанный текст:", result["text"])

    except KeyboardInterrupt:
        print("\nОстановка программы...")


Началась запись. Нажмите Ctrl+C для остановки.
Обработка аудио...


C:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст:  Если ваш ответ отразительен, то вы упали в нужное место.
Обработка аудио...
📝 Распознанный текст:  место. На линг вот...
Обработка аудио...
📝 Распознанный текст:  На Lingoa.com есть тексты на русском языке.
Обработка аудио...
📝 Распознанный текст:  на русском языке.
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст:  Опа, так.
Обработка аудио...
📝 Распознанный текст:  Задержка идет, да?
Обработка аудио...
📝 Распознанный текст:  короче я 